# Hand Labelled Homography
Generate a pipeline for quantitative analysis of homography estimation. Do
 - Randomly sample sequences from list of videos
 - Safe sequences into folder
 - Annotate some sequences
 - Create evaluation pipeline, precision, drift


In [4]:
import sys
import pandas as pd
from dotmap import DotMap

sys.path.append('../')

from utils.io import load_yaml

server = 'local'
servers = load_yaml('../config/servers.yml')
server = DotMap(servers[server])

meta_df = pd.read_pickle('../config/cholec80_transforms.pkl')
meta_df

,database,train,file,pre_transforms,aug_transforms,auxiliary
0,cholec80,True,"{'name': 'video01.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
1,cholec80,True,"{'name': 'video02.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
2,cholec80,True,"{'name': 'video03.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
3,cholec80,True,"{'name': 'video04.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
4,cholec80,True,"{'name': 'video05.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
...,...,...,...,...,...,...
70,cholec80,True,"{'name': 'video17.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
71,cholec80,True,"{'name': 'video18.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
72,cholec80,True,"{'name': 'video19.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
73,cholec80,True,"{'name': 'video20.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}


## Randomly Sample Image Sequences

In [2]:
import cv2
import numpy as np
from kornia import tensor_to_image

from utils.transforms import anyDictListToCompose
from utils.sampling import RandomSequences
from utils.io import generate_path

debug = False

max_seq = 5
paths = meta_df.apply(lambda x: os.path.join(server.database.location, x.database, x.file['path'], x.file['name']), axis=1).tolist()
seq_len = 50
strides = [1]

# append to tensor transform, as meta_df is supposed to operate on tensors
to_tensor = {'module': 'torchvision.transforms', 'type': 'ToTensor', 'kwargs': {}}
transforms = meta_df.apply(lambda x: anyDictListToCompose([to_tensor] + x.pre_transforms), axis=1).tolist()

random_sequences = RandomSequences(
    max_seq=max_seq,
    paths=paths,
    seq_len=seq_len,
    transforms=transforms,
    verbose=True
)

out_prefix = 'out/homography_labelling'

for seq, vid_idx, frame_idx in random_sequences:
    print('vid_idx: {}, frame_idx: {}'.format(vid_idx, frame_idx))
    for idx, frame in enumerate(seq):
        frame = (tensor_to_image(frame)*255).astype(np.uint8)
        if debug:
            cv2.imshow('random_frame', frame)  # show images
            cv2.waitKey()
        else:
            vid_path = os.path.join(out_prefix, 'vid_{}'.format(vid_idx))
            generate_path(vid_path)
            cv2.imwrite(os.path.join(vid_path, 'frame_{}.png'.format(frame_idx + idx*strides[0])), frame)
cv2.destroyAllWindows()

vid_idx: 7, frame_idx: 18640
vid_idx: 64, frame_idx: 33922
vid_idx: 71, frame_idx: 28329
vid_idx: 43, frame_idx: 6739
vid_idx: 26, frame_idx: 108610


### Dataset on Sampled Frames

In [ ]:
import os

from utils.io import recursive_scan2df

prefix = 'out/homography_labelling'

# create a simplified dataframe
postfix = '.png'
df = recursive_scan2df(prefix, postfix)
df['vid'] = df.folder.apply(lambda x: int(x.split('_')[-1]))
df['frame'] = df.file.apply(lambda x: int(x.split('_')[-1].replace(postfix, '')))
df = df.sort_values(['vid', 'frame']).reset_index(drop=True)

out_path = 'light_log'
df.to_pickle(os.path.join(prefix, out_path + '.pkl'))

df

## Estimate Homographies on Sequences

### Forward-Backward Consistency

In [6]:
import cv2
import numpy as np
import pandas as pd
from kornia import tensor_to_image
from torch.utils.data import DataLoader
import sys

sys.path.append('../')

from utils.transforms import anyDictListToCompose
from utils.processing import forward_backward_sequence
from datasets import ImageSequenceDataset

prefix = 'out/homography_labelling'
df = pd.read_pickle(os.path.join(prefix, 'light_log.pkl'))
seq_len = 10
transforms = meta_df.apply(lambda x: anyDictListToCompose(x.pre_transforms), axis=1).tolist()

ds = ImageSequenceDataset(
    df=df,
    prefix=prefix,
    seq_len=seq_len,
    transforms=transforms
)

# load network


batch_size = 1
num_workers = 0

dl = DataLoader(ds, batch_size=batch_size, num_workers=num_workers)

for batch in dl:
    fw, bw = forward_backward_sequence(batch)

torch.Size([1, 10, 3, 480, 640])
torch.Size([1, 6, 3, 480, 640]) torch.Size([1, 6, 3, 480, 640])
torch.Size([1, 10, 3, 480, 640])
torch.Size([1, 6, 3, 480, 640]) torch.Size([1, 6, 3, 480, 640])
torch.Size([1, 10, 3, 480, 640])
torch.Size([1, 6, 3, 480, 640]) torch.Size([1, 6, 3, 480, 640])
torch.Size([1, 10, 3, 480, 640])
torch.Size([1, 6, 3, 480, 640]) torch.Size([1, 6, 3, 480, 640])
torch.Size([1, 10, 3, 480, 640])
torch.Size([1, 6, 3, 480, 640]) torch.Size([1, 6, 3, 480, 640])
torch.Size([1, 10, 3, 480, 640])
torch.Size([1, 6, 3, 480, 640]) torch.Size([1, 6, 3, 480, 640])
torch.Size([1, 10, 3, 480, 640])
torch.Size([1, 6, 3, 480, 640]) torch.Size([1, 6, 3, 480, 640])
torch.Size([1, 10, 3, 480, 640])
torch.Size([1, 6, 3, 480, 640]) torch.Size([1, 6, 3, 480, 640])
torch.Size([1, 10, 3, 480, 640])
torch.Size([1, 6, 3, 480, 640]) torch.Size([1, 6, 3, 480, 640])
torch.Size([1, 10, 3, 480, 640])
torch.Size([1, 6, 3, 480, 640]) torch.Size([1, 6, 3, 480, 640])
torch.Size([1, 10, 3, 480, 640